In [1]:
from datasets import load_dataset
import torch
import torch.nn as nn
import transformers
from accelerate import PartialState
from datasets import Dataset, IterableDataset
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
local_ranks = 0

model_path = "/data/public/model/Qwen2.5-1.5B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_path)
# model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.bfloat16)
# model.to(local_ranks)
ds= load_dataset("/data/gwy/datasets/Bespoke-Stratos-17k")['train']


/data/gwy/miniconda3/envs/openr1/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 16710 examples [00:01, 14788.35 examples/s]


In [2]:
# 假设我们有一个列表，包含多个提示文本
prompts = ["prompt1", "prompt2", "prompt3", "prompt4", "prompt5", "prompt6", "prompt7", "prompt8", "prompt9", "prompt10"]

# 假设当前进程的索引是0
process_index = 0

# 创建一个slice对象
process_slice = slice(
    process_index * len(prompts),
    (process_index + 1) * len(prompts),
)

# 使用slice对象来选择列表中的一部分
selected_prompts = prompts[process_slice]

# 打印选择的提示文本
print(selected_prompts)


['prompt1', 'prompt2', 'prompt3', 'prompt4', 'prompt5', 'prompt6', 'prompt7', 'prompt8', 'prompt9', 'prompt10']


In [3]:
example = ds[0]
example

{'system': "Your role as an assistant involves thoroughly exploring questions through a systematic long thinking process before providing the final precise and accurate solutions. This requires engaging in a comprehensive cycle of analysis, summarizing, exploration, reassessment, reflection, backtracing, and iteration to develop well-considered thinking process. Please structure your response into two main sections: Thought and Solution. In the Thought section, detail your reasoning process using the specified format: <|begin_of_thought|> {thought with steps separated with '\\n\\n'} <|end_of_thought|> Each step should include detailed considerations such as analisying questions, summarizing relevant findings, brainstorming new ideas, verifying the accuracy of the current steps, refining any errors, and revisiting previous steps. In the Solution section, based on various attempts, explorations, and reflections from the Thought section, systematically present the final solution that you 

In [10]:
def split_prompt_and_responses_SafeRLHF(examples):
    messages = []
    for conv in examples['conversations']:
        if conv['from']=='user':
            messages.append({"role":'user', "content":conv['value']})
        else:
            messages.append({"role":'assistant', "content":conv['value']})

    return {
        "messages": messages ,
        "system": examples['system']
    }
ds = ds.map(split_prompt_and_responses_SafeRLHF, remove_columns=ds.column_names, num_proc=8)

Map (num_proc=8): 100%|██████████| 16710/16710 [00:01<00:00, 9645.13 examples/s] 


In [13]:
import json

output  = []
for data in ds:
    output.append(data)

In [14]:
with open("output.json", 'w') as file:
    json.dump(output, file, indent=4)

In [15]:
ds= load_dataset("output")

Generating train split: 16710 examples [00:02, 8107.69 examples/s]


In [17]:
ds['train']

Dataset({
    features: ['system', 'messages'],
    num_rows: 16710
})

In [36]:
from trl.data_utils import is_conversational, maybe_apply_chat_template

dataset = ds.map(
    maybe_apply_chat_template,
    fn_kwargs={"tokenizer": tokenizer},
    remove_columns="messages" if "messages" in ds.column_names else None,  # renamed to "text"
    num_proc = 6,
)

Map (num_proc=6): 100%|██████████| 16710/16710 [00:02<00:00, 7432.97 examples/s] 


In [37]:
dataset[0]

{'text': "<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\nReturn your final response within \\boxed{}. The operation $\\otimes$ is defined for all nonzero numbers by $a\\otimes b =\\frac{a^{2}}{b}$. Determine $[(1\\otimes 2)\\otimes 3]-[1\\otimes (2\\otimes 3)]$.\n$\\text{(A)}\\ -\\frac{2}{3}\\qquad\\text{(B)}\\ -\\frac{1}{4}\\qquad\\text{(C)}\\ 0\\qquad\\text{(D)}\\ \\frac{1}{4}\\qquad\\text{(E)}\\ \\frac{2}{3}$<|im_end|>\n<|im_start|>assistant\n<|begin_of_thought|>\n\nOkay, let me try to figure out this problem. So, we have this operation defined as a⊗b = a²/b. And we need to compute [(1⊗2)⊗3] - [1⊗(2⊗3)]. Then choose the correct answer from the options given. Alright, let's break it down step by step.\n\nFirst, I need to remember that the operation ⊗ is not associative, right? Because the problem is asking for the difference between two different groupings: (1⊗2)⊗3 and 1⊗(2⊗3). So, the order in which we perform th